# This file uses the run generated by `scenario_hsi_in_typical_run.py` and floweaver
to produce a Sankey diagram that maps appointments with HSI events.

Below is the instruction to run the file.

### Install floweaver in Anaconda Prompt (if use Jupyter Notebook) / PyCharm Terminal:

pip install floweaver

pip install ipysankeywidget

jupyter nbextension enable --py --sys-prefix widgetsnbextension

jupyter nbextension enable --py --sys-prefix ipysankeywidget

jupyter notebook (to open jupyter notebook)

### To display and save the output figures:
Select Start Jupyter Server from the Jupyter Actions Menu (lightbulb icon next to Run All cells icon)

Open Event Log

Open in Browser

Find the script and run all cells


In [ ]:
import tlo

from pathlib import Path

from tlo.analysis.utils import get_scenario_outputs, load_pickled_dataframes

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt

from ipysankeywidget import SankeyWidget

from floweaver import *

In [ ]:
# Declare the name of the file that specified the scenarios used in this run.
scenario_filename = '10_year_scale_run.py'

# Declare usual paths:
# Get the tlo path
tlopath = Path(tlo.__file__).parent.parent.parent
outputspath = tlopath / Path('outputs/bshe@ic.ac.uk')

# Find results folder (most recent run generated using that scenario_filename)
results_folder = get_scenario_outputs(scenario_filename, outputspath)[-4]
print(f"Results folder is: {results_folder}")

# Declare path for output graphs from this script
make_graph_file_name = lambda stub: results_folder / f"{stub}.png"  # noqa: E731

# Extract results
hcw_time = pd.read_csv(results_folder / 'hcw_working_time_per_hsi.csv')

# Format data for flow
hcw_time['Module'] = hcw_time['Treatment_ID'].str.split('_').apply(lambda x: x[0])
hcw_time = hcw_time.groupby(['Officer_Category', 'Appt_Category', 'Module'])['Total_Mins_Used_Per_Year'].sum().reset_index()

hcw_time['source'] = 'Officer_Category'
hcw_time['target'] = 'Module'
hcw_time['value'] = hcw_time['Total_Mins_Used_Per_Year']

# Format the flow

partition_officer_cat = Partition.Simple('Officer_Category',
                                         pd.array(['DCSA', 'Clinical', 'Nursing and Midwifery', 'Pharmacy',
                                                   'Laboratory', 'Radiography', 'Mental']))
partition_module = Partition.Simple('Module',
                                   np.unique(hcw_time['Module']))
partition_appt_cat = Partition.Simple('Appt_Category',
                                      pd.array(['ConWithDCSA', 'IPOP', 'RMNCH', 'MISC',
                                      'HIV', 'TB', 'NUTRITION', 'PharmDispensing', 'LABORATORY',
                                      'RADIOGRAPHY', 'MENTAL']))


nodes = {
    'Officer': ProcessGroup(['Officer_Category'], partition_officer_cat),
    'Module': ProcessGroup(['Module'], partition_module),
}

# Add nodes Waypoint
nodes['waypoint'] = Waypoint(partition_appt_cat)

bundles = [
    Bundle('Officer', 'Module', waypoints=['waypoint']),
]

ordering = [
    ['Officer'],  # left
    ['waypoint'],    # middle
    ['Module'],     # right
    ]

# Set the color for each officer category
palette = {'Clinical': 'skyblue', 'Nursing and Midwifery': 'lightpink',
           'Pharmacy': 'khaki', 'Laboratory': 'cadetblue',
           'Radiography': 'yellowgreen',
           'Mental': 'mediumorchid', 'DCSA': 'royalblue'
          }

# Set the size for the Sankey
size = dict(width=800, height=800, margins=dict(left=180, right=180))

# Sankey diagram definition (SDD)
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_officer_cat)

hcw_time_flow = weave(sdd, hcw_time, palette=palette, measures='value').to_widget(**size)

hcw_time_flow.auto_save_png(results_folder /'Sankey_hcw_time_flow.png')
